In [1]:
import sys, os, git
import shutil
import numpy as np
import subprocess

from utilities import detect_saving_folder, try_to_copy_file, \
                      return_file_content, detect_block_code, \
                      detect_block_types, create_file, append_content
                      

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")

path_to_docs = git_path + "/docs/source/chapters/"

# make sure the documentaiton was found
assert os.path.exists(path_to_docs), """Documentation files not found"""

if os.path.exists("generated-codes/") is False:
    os.mkdir("generated-codes/")

In [49]:
import os
import numpy as np
import shutil

def detect_saving_folder(chapter_id, CREATE=True):
    folder = "generated-codes/chapter"+str(chapter_id)+"/"
    if CREATE:
        if os.path.exists(folder) is False:
            os.mkdir(folder)
    return folder

def return_file_content(filename):
    file = open(filename, "r")
    file_content = []
    for line in file:
        file_content.append(line)
    file.close()
    return file_content

def detect_block_code(file_content):
    block_contents = []
    block_names = []
    LABEL = False
    for line in file_content:
        if ".. label::" in line: # Detect the label "start" and label "end"
            label = line.split(".. label:: ")[1] # Look for label in the line
            if label[:6] == "start_": # Detect starting label
                class_name_i = label.split("start_")[1].split("_class")[0]
                block = []
                LABEL = True
            elif label[:4] == "end_": # Detect ending label
                class_name_f = label.split("end_")[1].split("_class")[0]
                LABEL = False
                block_contents.append(block)
                assert class_name_i == class_name_f
                block_names.append(class_name_i)
        else:
            if LABEL: # Print the content of the label into files
                if ".. code-block::" not in line: # Ignore code block line
                    if len(line) > 1: # Remove the indentation
                        block.append(line[4:])
                    else:
                        block.append(line)
    return block_contents, block_names

def detect_block_types(block_contents):
    block_types = []
    for block in block_contents:
        ISIMPORT = False
        ISMETHOD = False
        ISCLASS = False
        ISPARTIAL = False
        for line in block:
            # look for "from X import Y" command
            if (("import" in line) & ("as" in line)) | (("from" in line) & ("import" in line)):
                ISIMPORT = True
            # look for "def X():" line
            if ("def" in line) & ("(self" in line) & ("):" in line):
                ISMETHOD = True
            # look for "class C():" line
            if ("class" in line) & (":" in line):
                ISCLASS = True
            # look for "(...)" line
            if ("(...)" in line):
                ISPARTIAL = True
        block_types.append([ISIMPORT, ISMETHOD, ISCLASS, ISPARTIAL])
    return block_types

def create_file(block_contents, block_names, created_files, created_tests, folder):
    for content, name in zip(block_contents, block_names):
        if name+".py" not in created_files:
            if "test_" not in name:
                created_files.append(name+".py")
                file = open(folder+name+".py", "w")
                for line in content:
                    file.write(line)
                file.close()
            else:
                created_tests.append(name+".py")
                file = open(folder+name+".py", "w")
                for line in content:
                    file.write(line)
                file.close()            
    return created_files, created_tests

def try_to_copy_file(chapter_id, created_files):
    new_folder = detect_saving_folder(chapter_id, CREATE=False)
    old_folder = detect_saving_folder(chapter_id-1, CREATE=False)
    if os.path.exists(old_folder):
        existing_files = next(os.walk(old_folder), (None, None, []))[2]
        for file in existing_files:
            if (".py" in file) and ("test_" not in file):
                shutil.copyfile(old_folder+"/"+file,
                                new_folder+"/"+file)
                created_files.append(file)
    return created_files

def detect_last_matching_line(content, original_file_content):
    for cpt_new, new_line in enumerate(content):
        if len(new_line) > 1:
            for cpt_old, older_line in enumerate(original_file_content):
                if len(older_line) > 0:
                    if new_line in older_line:
                        cpt_new_last = cpt_new
                        cpt_old_last = cpt_old
    return cpt_new_last, cpt_old_last

def detect_method_boundaries(method_name, original_file_content):
    original_start_init = None
    original_end_init = []
    last_line = None
    for cpt, l in enumerate(original_file_content):
        if ("def" in l) & (method_name in l):
            original_start_init = cpt
        elif (":" in l) & ("def" in l) & (method_name not in l):
            original_end_init.append(cpt)     
        if len(l) > 1:
            last_line = cpt
        if (len(l) > 1) & (original_start_init is None):
            original_start_init = cpt
    if len(original_end_init) > 0:  
        original_end_init = original_end_init[0]
    else:
        original_end_init = last_line
    if original_end_init < original_start_init:
        original_end_init = last_line
    return original_start_init, original_end_init+1

def detect_unique_lines(file_content, start, end):
    _, idx = np.unique(file_content[start:end],
                       return_index=True)
    unique_lines = []
    for i in np.sort(idx):
        unique_lines.append(file_content[start:end][i])
    return unique_lines

def replace_method(folder, name, original_content, unique_lines, original_start, original_end):
    REPLACED = False
    new_class = open(folder+name+".py", "w")
    for cpt, l in enumerate(original_content):
        if (cpt < original_start) | (cpt > original_end-2):
            new_class.write(l)
        else:
            if REPLACED is False:
                REPLACED = True
                for ll in unique_lines:
                    new_class.write(ll)
    new_class.close()

def detect_methods(file_content):
    existing_methods = []
    for line in file_content:
        if ("def " in line) & ("(self" in line):
            method_name = line.split("def ")[1].split("(self")[0]
            existing_methods.append(method_name)
    return existing_methods

def detect_existing_lines(ncontent, ocontent):
    list_of_empty_line = ["\n"]
    for _ in np.arange(20):
        list_of_empty_line.append(" "+list_of_empty_line[-1])
    status = []
    for ncpt, nl in enumerate(ncontent):
        if nl in list_of_empty_line:
            status.append(["LINE IS EMPTY", ncpt, None])
        else:
            FOUND = False
            for ocpt, ol in enumerate(ocontent):
                if len(ol) > 1:
                    if nl in ol:
                        FOUND = True
                        oloc = ocpt
            if FOUND:
                status.append(["LINE DOES EXIST", ncpt, oloc])
            else:
                if "(...)" in nl:
                    status.append(["TRANSITION", ncpt, None])
                else:
                    status.append(["LINE DOESNT EXIST", ncpt, None])

    return np.array(status)

def write_non_method(new_content, original_file_content, folder, name, indent):
    # check if the new lines are new or not
    status_new_lines = detect_existing_lines(new_content,
                                             original_file_content)   
    # The new content is "import" function, to be added at the
    # beginning of the file
    file = open(folder+name+".py", "w")
    for line, status_line in zip(new_content, status_new_lines):
        status, pos_new, pos_old = status_line
        if "LINE DOESNT EXIST" in status: # only add non empty line that don't exists 
            file.write(indent+line)
    for line in original_file_content:
        file.write(line)
    file.close()

def write_new_method(new_content, original_file_content, folder, name, indent, status_new_lines):
    file = open(folder+name+".py", "w")
    for line in original_file_content:
        file.write(line)
    file.write("\n")
    for line, status_line in zip(new_content, status_new_lines):
        status, pos_new, pos_old = status_line
        if "LINE DOESNT EXIST" in status: # only add non empty line that don't exists 
            file.write(indent+line)
    file.close()

def append_new_line_to_method(new_content, original_file_content, folder, name,
                              indent, status_new_lines, original_start, LAST_TRANSITION = False):
    if LAST_TRANSITION:
        transition_id = np.where(status_new_lines[:,0] == 'TRANSITION')[0][-1]
    else:
        transition_id = np.where(status_new_lines[:,0] == 'TRANSITION')[0]
    new_line_id = np.where(status_new_lines[:,0] == 'LINE DOESNT EXIST')[0]
    last_exising_line = np.max(status_new_lines[status_new_lines[:,0] == 'LINE DOES EXIST'][:,2])
    last_exising_line += 1
    if True in np.unique(new_line_id > transition_id):
        # the new line are after the transition
        # new content to be placed at the end of the method
        file = open(folder+name+".py", "w")
        for line in original_file_content[:original_start+last_exising_line]:
            file.write(line)
        for line, status_line in zip(new_content,
                                    status_new_lines):
            status, pos_new, pos_old = status_line
            if "LINE DOESNT EXIST" in status: # only add non empty line that don't exists 
                file.write(indent+line)
        for line in original_file_content[original_start+last_exising_line:]:
            file.write(line)
        file.close()
    else:
        print("that possibility was not anticipated")

def append_new_line_no_transition(new_content, original_file_content, folder, name, indent, status_new_lines, original_start):
    file = open(folder+name+".py", "w")
    for cpt, line in enumerate(original_file_content):
        file.write(line)
        if cpt-original_start in status_new_lines[:,2]:
            STOP = True
            for new_line, status_line in zip(new_content, status_new_lines):
                status, pos_new, pos_old = status_line
                try:
                    if pos_old == cpt-original_start:
                        STOP = False
                    elif pos_old > cpt-original_start:
                        STOP = True
                except:
                    assert pos_old is None
                if ("LINE DOESNT EXIST" in status) & (pos_old is None) & (STOP is False): # only add non empty line that don't exists 
                    file.write(indent+new_line)
    file.close()    

In [51]:
mycwd = os.getcwd()
for chapter_id in np.arange(3):
    filename = path_to_docs + "chapter"+str(chapter_id)+".rst"
    created_files, created_tests = [], []
    if os.path.exists(filename):
        folder = detect_saving_folder(chapter_id)
        created_files = try_to_copy_file(chapter_id, created_files)
        file_content = return_file_content(filename)
        block_contents, block_names = detect_block_code(file_content)
        block_types = detect_block_types(block_contents)
        created_files, created_tests = create_file(block_contents, block_names,
                                                   created_files, created_tests, folder)
        
        
        for new_content, name, type in zip(block_contents, block_names, block_types):
            #append_content(folder, name, new_content, type)
            
            
            # Run the tests
            # os.chdir(folder)
            # for test_file in created_tests:
            #     print("TEST --", "chapter"+str(chapter_id)+".rst", "--", test_file)
            #     subprocess.call(["python3", test_file])
            # os.chdir(mycwd)

            #def append_content(folder, name, new_content, type):
            #    if "test_" not in name:
            ISIMPORT, ISMETHOD, ISCLASS, ISPARTIAL = type
            indent = ""
            if np.sum(type) > 0: # nothing to append
                original_file_content = return_file_content(folder+name+".py")
                existing_methods = detect_methods(original_file_content)
                new_method = detect_methods(new_content)

                if len(new_method) == 0: # the added block is not a method
                    assert ISIMPORT
                    write_non_method(new_content, original_file_content, folder, name, indent)
                elif len(new_method) == 1: # the added block is a method

                    # if the class is not specified in the doc,
                    # the intend will be wrong
                    if ISCLASS is False:
                        indent = "    "

                    original_start, original_end = detect_method_boundaries(new_method[0],
                                                                            original_file_content)
                    new_start, new_end = detect_method_boundaries(new_method[0],
                                                                  new_content)
                    new_end += 1
                    # detect the line that are in both original and readded file
                    status_new_lines = detect_existing_lines(new_content[new_start:new_end],
                                                             original_file_content[original_start:original_end])   
                    
                    NOTHING_TO_ADD = False
                    if (len(np.unique(status_new_lines[:,0])) == 1):
                        if ("LINE DOES EXIST" in status_new_lines[:,0]) | ("LINE IS EMPTY" in status_new_lines[:,0]):
                            NOTHING_TO_ADD = True
                    if (len(np.unique(status_new_lines[:,0])) == 2):
                        if ("LINE DOES EXIST" in status_new_lines[:,0]) & ("LINE IS EMPTY" in status_new_lines[:,0]):
                            NOTHING_TO_ADD = True

                    if NOTHING_TO_ADD == False:
                        if new_method[0] not in existing_methods:
                            # this is a new method, to be added at the end
                            write_new_method(new_content[new_start:new_end], original_file_content,
                                            folder, name, indent, status_new_lines)
                        else:                            
                            if np.sum(status_new_lines[:,0] == 'TRANSITION') == 1:
                                # there is one transition (...) in new_content
                                append_new_line_to_method(new_content[new_start:new_end], original_file_content,
                                                        folder, name, indent, status_new_lines, original_start)
                            elif np.sum(status_new_lines[:,0] == 'TRANSITION') == 0:
                                # There if no transition, but content must be added to existing method
                                append_new_line_no_transition(new_content[new_start:new_end], original_file_content,
                                                              folder, name, indent, status_new_lines, original_start)
                            else:
                                append_new_line_to_method(new_content[new_start:new_end], original_file_content,
                                                        folder, name, indent, status_new_lines,
                                                        original_start, LAST_TRANSITION=True)
                else:
                    print("NOT ANTICIPATED: SEVERAL METHODS")

In [36]:
transition_id = np.where(status_new_lines[:,0] == 'TRANSITION')[0]
new_line_id = np.where(status_new_lines[:,0] == 'LINE DOESNT EXIST')[0]
last_exising_line = np.max(status_new_lines[status_new_lines[:,0] == 'LINE DOES EXIST'][:,2])
transition_id, new_line_id, last_exising_line+original_start

(array([1]), array([3]), 16)

In [41]:

if True in np.unique(new_line_id > transition_id):
    # the new line are after the transition
    # new content to be placed at the end of the method
    #file = open(folder+name+".py", "w")
    for line in original_file_content[:original_start+last_exising_line+1]:
        print(line[:-1])
    for line, status_line in zip(new_content,
                                status_new_lines):
        status, pos_new, pos_old = status_line
        if "LINE DOESNT EXIST" in status: # only add non empty line that don't exists 
            print(indent+line)
            pass
    for line in original_file_content[original_start+last_exising_line+1:]:
        #file.write(line)
        print(line)
    #file.close()

import numpy as np
from scipy import constants as cst


class Prepare:
    def __init__(self,
                number_atoms=[10],  # List
                epsilon=[0.1],  # List - Kcal/mol
                sigma=[1],  # List - Angstrom
                atom_mass=[1],  # List - g/mol
                *args,
                **kwargs):
        self.number_atoms = number_atoms
        self.epsilon = epsilon
        self.sigma = sigma
        self.atom_mass = atom_mass
        super().__init__(*args, **kwargs)
        (...)





    def calculate_LJunits_prefactors(self):

        # Distance, energy, and mass

        self.reference_distance = self.sigma[0]  # Angstrom

        self.reference_energy = self.epsilon[0]  # Kcal/mol

        self.reference_mass = self.atom_mass[0]  # g/mol

        # Time

        mass_kg = self.atom_mass[0]/cst.kilo/cst.Avogadro  # kg

        epsilon_J = self.epsilon[0]*cst.calorie*cst.kilo/cst.Avogadro  # J

        sigma_m = self.sigma[0]*cst.angstrom  # m

    

In [31]:
new_content[new_start:new_end]

['def __init__(self,\n',
 '    (...)\n',
 '    super().__init__(*args, **kwargs)\n',
 '    self.calculate_LJunits_prefactors()\n']

In [6]:
new_content[new_start:new_end]

['import numpy as np\n',
 'from Prepare import Prepare\n',
 '\n',
 'prep = Prepare(number_atoms=[2, 3],\n',
 '    epsilon=[0.1, 1.0], # kcal/mol\n',
 '    sigma=[3, 6], # A\n']

In [7]:
status_new_lines

array([['LINE DOES EXIST', 0, 0],
       ['TRANSITION', 1, None],
       ['LINE DOES EXIST', 2, 13],
       ['LINE DOES EXIST', 3, 11],
       ['LINE DOES EXIST', 4, 12],
       ['LINE DOESNT EXIST', 5, None]], dtype=object)

In [8]:
new_content[new_start:new_end]

['import numpy as np\n',
 'from Prepare import Prepare\n',
 '\n',
 'prep = Prepare(number_atoms=[2, 3],\n',
 '    epsilon=[0.1, 1.0], # kcal/mol\n',
 '    sigma=[3, 6], # A\n']

In [9]:
last_exising_line

NameError: name 'last_exising_line' is not defined

['    def __init__(self,\n',
 '                number_atoms=[10],  # List\n',
 '                epsilon=[0.1],  # List - Kcal/mol\n',
 '                sigma=[1],  # List - Angstrom\n',
 '                atom_mass=[1],  # List - g/mol\n',
 '                *args,\n',
 '                **kwargs):\n',
 '        self.number_atoms = number_atoms\n',
 '        self.epsilon = epsilon\n',
 '        self.sigma = sigma\n',
 '        self.atom_mass = atom_mass\n',
 '        super().__init__(*args, **kwargs)\n']

In [ ]:
status_new_lines

array([['LINE DOES EXIST', 0, 0],
       ['LINE DOES EXIST', 1, 1],
       ['LINE DOES EXIST', 2, 2],
       ['LINE DOES EXIST', 3, 3],
       ['LINE IS EMPTY', 4, None]], dtype=object)

In [ ]:


        
        


                    
                else:
                    lines_to_add = []
                    last_old_line = -1
                    for status_line, content in zip(status_new_lines, new_content[new_start:new_end]):
                        status, nl, ol = status_line
                        if ol is not None:
                            last_old_line = ol
                        if status == "LINE DOESNT EXIST":
                            lines_to_add.append([content, last_old_line+original_start])
                    lines_to_add = np.array(lines_to_add)
                    if len(lines_to_add)>0:
                        if len(np.unique(np.int32(lines_to_add[:, 1]))) == 1:
                            if np.unique(np.int32(lines_to_add[:, 1])) == -1:
                                # Completely new method, should be added at the end                 
                                file = open(folder+name+".py", "w")
                                for line in original_file_content:
                                    file.write(line)
                                file.write(" ")
                                for line, status_line in zip(new_content[new_start:new_end], status_new_lines):
                                    status, pos_new, pos_old = status_line
                                    if "LINE DOESNT EXIST" in status: # only add non empty line that don't exists 
                                        file.write(indent+line)
                                file.close()
                            else:
                                print("what happens here ?")
                        else:
                            file = open(folder+name+".py", "w")
                            for cpt, line in enumerate(original_file_content):
                                file.write(line)
                                if cpt in np.int32(lines_to_add[:, 1]):
                                    for line in lines_to_add[cpt == np.int32(lines_to_add[:, 1])][:,0]:
                                        file.write(line) 
                            file.close()

In [ ]:
original_file_content = return_file_content(folder+name+".py")
existing_methods = detect_methods(original_file_content)
new_method = detect_methods(new_content)

original_start, original_end = detect_method_boundaries(new_method[0],
                                                        original_file_content)
new_start, new_end = detect_method_boundaries(new_method[0], new_content)
# detect the line that are in both original and readded file
status_new_lines = detect_existing_lines(new_content[new_start:new_end],
                                            original_file_content[original_start:original_end])   




TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'

array(['                number_atoms=[10],  # List\n',
       '                epsilon=[0.1],  # List - Kcal/mol\n',
       '                sigma=[1],  # List - Angstrom\n',
       '                atom_mass=[1],  # List - g/mol\n'], dtype='<U50')

In [ ]:
 if "test_" not in name:
    ISIMPORT, ISMETHOD, ISCLASS, ISPARTIAL = type
    indent = ""
    if np.sum(type) == 0: # nothing to append
        pass
    else:
        original_file_content = return_file_content(folder+name+".py")
        existing_methods = detect_methods(original_file_content)
        new_method = detect_methods(new_content)

        if len(new_method) == 0: # the added block is not a method
            assert ISIMPORT
            # check if the new lines are new or not
            status_new_lines = detect_existing_lines(new_content,
                                                        original_file_content)   
            # The new content is "import" function, to be added at the
            # beginning of the file
            file = open(folder+name+".py", "w")
            for line, status_line in zip(new_content, status_new_lines):
                status, pos_new, pos_old = status_line
                if "LINE DOESNT EXIST" in status: # only add non empty line that don't exists 
                    file.write(indent+line)
            for line in original_file_content:
                file.write(line)
            file.close()

        else: # the added block is a method
            original_start, original_end = detect_method_boundaries(new_method[0], original_file_content)
            new_start, new_end = detect_method_boundaries(new_method[0], new_content)
            # detect the line that are in both original and readded file
            status_new_lines = detect_existing_lines(new_content[new_start:new_end],
                                                        original_file_content[original_start:original_end])   

            # if the class is not specified in the doc,
            # the intend will be wrong
            if ISCLASS is False:
                indent = "    "

            if np.sum(status_new_lines[:,0] == 'TRANSITION'): # there is one transition (...)

                transition_id = np.where(status_new_lines[:,0] == 'TRANSITION')[0]
                new_line_id = np.where(status_new_lines[:,0] == 'LINE DOESNT EXIST')[0]

                if True in np.unique(new_line_id > transition_id):
                    # the new line are after the transition
                    # new content to be placed at the end of the method
                    file = open(folder+name+".py", "w")
                    for line in original_file_content[:original_end]:
                        file.write(line)
                    for line, status_line in zip(new_content[new_start:new_end], status_new_lines):
                        status, pos_new, pos_old = status_line
                        if "LINE DOESNT EXIST" in status: # only add non empty line that don't exists 
                            file.write(indent+line)
                    for line in original_file_content[original_end:]:
                        file.write(line)
                    file.close()

                else:
                    print("that possibility was not anticipated")

            else:

                print()

In [ ]:
        for new_content, name, type in zip(block_contents, block_names, block_types):
            if "test_" not in name:
                ISIMPORT, ISMETHOD, ISCLASS, ISPARTIAL = type
                indent = ""
                if np.sum(type) == 0: # nothing to append
                    pass
                else:
                    original_file_content = return_file_content(folder+name+".py")
                    existing_methods = detect_methods(original_file_content)
                    new_method = detect_methods(new_content)

                    if len(new_method) == 0: # the added block is not a method
                        assert ISIMPORT
                        # check if the new lines are new or not
                        status_new_lines = detect_existing_lines(new_content,
                                                                 original_file_content)   
                        # The new content is "import" function, to be added at the
                        # beginning of the file
                        file = open(folder+name+".py", "w")
                        for line, status_line in zip(new_content, status_new_lines):
                            status, pos_new, pos_old = status_line
                            if "LINE DOESNT EXIST" in status: # only add non empty line that don't exists 
                                file.write(indent+line)
                        for line in original_file_content:
                            file.write(line)
                        file.close()

                    else: # the added block is a method
                        original_start, original_end = detect_method_boundaries(new_method[0], original_file_content)
                        new_start, new_end = detect_method_boundaries(new_method[0], new_content)
                        # detect the line that are in both original and readded file
                        status_new_lines = detect_existing_lines(new_content[new_start:new_end],
                                                                 original_file_content[original_start:original_end])   

                        # if the class is not specified in the doc,
                        # the intend will be wrong
                        if ISCLASS is False:
                            indent = "    "
            
                        if np.sum(status_new_lines[:,0] == 'TRANSITION'): # there is one transition (...)

                            transition_id = np.where(status_new_lines[:,0] == 'TRANSITION')[0]
                            new_line_id = np.where(status_new_lines[:,0] == 'LINE DOESNT EXIST')[0]

                            if True in np.unique(new_line_id > transition_id):
                                # the new line are after the transition
                                # new content to be placed at the end of the method
                                file = open(folder+name+".py", "w")
                                for line in original_file_content[:original_end]:
                                    file.write(line)
                                for line, status_line in zip(new_content[new_start:new_end], status_new_lines):
                                    status, pos_new, pos_old = status_line
                                    if "LINE DOESNT EXIST" in status: # only add non empty line that don't exists 
                                        file.write(indent+line)
                                for line in original_file_content[original_end:]:
                                    file.write(line)
                                file.close()

                            else:
                                print("that possibility was not anticipated")


that possibility was not anticipated


In [ ]:
new_line_id > transition_id

array([], dtype=bool)

In [ ]:
new_line_id

array([], dtype=int64)

In [ ]:
transition_id

array([1])

In [ ]:
new_content

['\n',
 '\n',
 'from MinimizeEnergy import MinimizeEnergy\n',
 '\n',
 'min = MinimizeEnergy(maximum_steps=100,\n',
 '    number_atoms=[2, 3],\n',
 '    epsilon=[0.1, 1.0], # kcal/mol\n',
 '    sigma=[3, 6], # A\n',
 '    atom_mass=[1, 1], # g/mol\n',
 '    box_dimensions=[20, 20, 20], # A\n',
 '    )\n',
 'min.run()\n',
 '\n',
 'assert min.compute_potential(output="potential") < 0\n',
 '\n']

In [ ]:
new_line_id

array([], dtype=int64)